In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import clodius.tiles.bam as ctb
import pysam

filename = '../data/SRR1770413.sorted.short.bam'
index_filename = '../data/SRR1770413.sorted.short.bam.bai'

f = pysam.AlignmentFile(filename, index_filename=index_filename)

In [19]:
reads = list(f.fetch('Chromosome', 1, 10000))

In [21]:
%%time

for read in reads[:150]:
    print(read.cigarstring)
    print([r for r in read.get_aligned_pairs(with_seq=True) if r[2] is not None and r[2].islower()])

167H134M
[(93, 93, 'a'), (98, 98, 'a')]
30S271M
[]
183H118M
[]
37S264M
[(299, 262, 'a')]
301M
[]
134S167M
[(195, 61, 'g')]
37S264M
[(246, 209, 'c'), (262, 225, 'a'), (269, 232, 'c'), (283, 246, 'g')]
153H148M
[]
252H49M
[]
229H72M
[]
224H77M
[]
51S250M
[(217, 166, 'a'), (267, 216, 'a'), (268, 217, 'c'), (279, 228, 'a')]
117S184M
[]
56S245M
[(183, 127, 'a')]
47S254M
[]
257H44M
[(43, 43, 'a')]
53S248M
[(289, 236, 'a'), (292, 239, 't')]
47S254M
[]
149S152M
[]
209H92M
[]
143S158M
[]
41S260M
[]
131S170M
[(137, 6, 't'), (153, 22, 'g')]
154H147M
[(112, 112, 'g')]
58S243M
[(116, 58, 't')]
92S209M
[]
57S244M
[(69, 12, 't')]
210H91M
[]
157H144M
[(81, 81, 't')]
229H72M
[]
26S275M
[(40, 14, 'a'), (54, 28, 't'), (64, 38, 't'), (187, 161, 'g'), (212, 186, 't'), (279, 253, 'g')]
87S214M
[]
263H38M
[]
172H129M
[]
103S198M
[]
9S292M
[]
124S177M
[]
47S254M
[]
267H34M
[]
92S209M
[(226, 134, 'a')]
131S170M
[]
236H65M
[]
8S293M
[]
301M
[(31, 32, 't')]
301M
[]
12S289M
[]
301M
[(291, 294, 't')]
301M
[(226, 2

In [31]:
reads[1].cigartuples

[(4, 30), (0, 271)]

In [30]:
reads[1].cigarstring

'30S271M'

301

In [55]:
def get_cigar_substitutions(read):
    subs = []
    curr_pos = 0
    
    cigartuples = read.cigartuples
    readstart = read.pos
    readend = read.pos + read.query_length
    
    for ctuple in cigartuples:
        if ctuple[0] == pysam.CDIFF:
            subs.append((None, readstart + curr_pos, 'X', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CINS:
            subs.append((None, readstart + curr_pos, 'I', ctuple[1]))
        elif ctuple[0] == pysam.CDEL:
            subs.append((None, readstart + curr_pos, 'D', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CREF_SKIP:
            subs.append((None, readstart + curr_pos, 'N', ctuple[1]))
            curr_pos += ctuple[1]
        elif ctuple[0] == pysam.CEQUAL or ctuple[0] == pysam.CMATCH:
            curr_pos += ctuple[1]
    
    if len(cigartuples):
        first_ctuple = cigartuples[0]
        last_ctuple = cigartuples[-1]
        
        if first_ctuple[0] == pysam.CSOFT_CLIP:
            subs.append((None, readstart-first_ctuple[1], 'S', first_ctuple[1]))
        if first_ctuple[0] == pysam.CHARD_CLIP:
            subs.append((None, readstart-first_ctuple[1], 'H', first_ctuple[1])) 

        if last_ctuple[0] == pysam.CSOFT_CLIP:
            subs.append((None, readend, 'S', last_ctuple[1]))
        if last_ctuple[0] == pysam.CHARD_CLIP:
            subs.append((None, readend, 'H', last_ctuple[1])) 

    return subs

In [57]:
get_cigar_substitutions(reads[1])

[(None, -30, 'S', 30)]